In [15]:
import astropy
import astropy.units as u
import astroquery
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import quantity_support
from astropy.time import Time 
import os
from astropy.io.votable import parse_single_table

In [34]:
# inputs: ra, dec, height_of_rectangle, width_of_rectangle, radius_of_circle
def psr_to_gaia(jname, raj, decj,  pmra, pmdec, posepoch, height, width):
    """Search Gaia for Possible Companion to Pulsar

    Given input parameters read in from a text file following the guidelines of ATNF parameters, 
    queries Gaia DR2 to find matches based on RA and Dec for each object from the text file

    Args:
        jname (str): Name of the pulsar being checked for matches 
        raj (str): Right ascension of the pulsar in hh:mm:ss.ss format -- is this actually what it is?
        decj (str): Declination of the pulsar in degrees:mm:ss.ss format -- is this actually what it is?
        pmra (): Proper motion in ra of the pulsar in -- format
        pmdec (): proper motion in dec of the pulsar in -- format
        height (str): Height of Gaia search in mas 
        width (str): Width of Gaia search in mas 
    
    Returns:
        Table: results of the Gaia query in an astropy Table  
    """

    #Import things

    # Given an item (via specifying ra/dec range or optionally inputing a pulsar name) from ATNF catalouge, 
    # find nearby Gaia objects
    from astropy.time import Time 
    p_ra = raj
    p_dec = decj
    p_pmra = pmra
    p_pmdec = pmdec
    p_epoch = Time(posepoch, format='mjd').jyear

    # convert ra and dec to angle objects that will know to behave as floats 
    p_ra_ang = Angle(p_ra, u.degree) # this is a potential error -- this takes the input as degrees, when
    # we should instatntiate the Quantity as what it is from atnf and convert to degrees if that is how 
    # we want them to be represented
    p_dec_ang = Angle(p_dec, u.degree) # yes, these are looking more and more suspicious to me

    # convert pmra and pmdec from mas/yr to deg/yr
    p_pmra_deg = (float(p_pmra) * u.mas).to(u.deg) / u.yr
    p_pmdec_deg = (float(p_pmdec) * u.mas).to(u.deg) / u.yr

    print(p_pmdec_deg)
    print(p_pmdec_deg.unit())

    # update location of pulsar based on difference from gaia epoch and pmra/pmdec
    gaia_epoch = 2015.5 * u.yr
    year_diff = gaia_epoch - p_epoch.tolist() * u.yr

    # get the new ra and dec for the pulsar by updating to gaia epoch 
    p_new_ra = p_ra_ang + (p_pmra_deg * year_diff)
    p_new_dec = p_dec_ang + (p_pmdec_deg * year_diff)

    # ra_ext_pos = raj + rajerr + (pmra + pmraerr)*year_diff
    # ra_ext_neg = raj - rajerr + (pmra - pmraerr)*year_diff

    # dec_ext_pos = decj + decjerr + (pmdec + pmdecerr)*year_diff
    # dec_ext_neg = decj - decjerr + (pmdec - pmdecerr)*year_diff


    # Query Gaia within the range of the given pulsar 
    coord=SkyCoord(ra=p_new_ra, dec=p_new_dec, unit=(u.degree, u.degree), frame='icrs')
    width_gaia = u.Quantity(width, u.arcmin)
    height_gaia = u.Quantity(height, u.arcmin)
    results = Gaia.query_object_async(coordinate=coord, width=width_gaia, height=height_gaia)
    
    if len(results) == 0:
        return results
    else:
        results.add_column(jname, name='Companion Pulsar', index=0)
        return results

In [6]:
def get_matches(input_file, output_file, height=1., width=1.):
    """Give Gaia matches to Pulsars 

    Takes as input a text file (.csv file) with index number, name, ra, dec, proper
    motion ra, proper motion dec and posepoch and produces all of the gaia matches
    of ra and dec to within a certain range.

    Args: 
        input_file (str): Name of the text file (csv) containing each pulsar with the parameters 'index', 'name', 
            'ra', 'dec', 'pmra', 'pmdec', 'posepoch' row by row for each object.
        output_file (str): Name of the text file which the pulsar-gaia matches will be output to.
        height (:obj:'float', optional): Height of the rectangle Gaia will query in.
        width (:obj:'float', optional): Width of the rectangle Gaia will query in.


    """
    from astropy.table import Table, vstack

    f = open(input_file, "r")
    results = Table()
    first_time = True

    counter = 0

    # Loop through file of ATNF data and combine tables of Gaia matches into one supertable
    for line in f:
      # Loop 20 times
      counter += 1
      if (counter == 20):
        break

      # Parse input
      values = line.split(';')

      # Add result to supertable
      search_result = psr_to_gaia(values[1],values[2],values[3],values[4],values[5],values[6], height, width)
      if (len(search_result) == 0):
        continue
      if first_time:
        results = search_result
        first_time = False
      else:  
        results = vstack([results, search_result])

    results.write(output_file, format='csv', overwrite=True)

Below is a number of unit tests on psr_to_gaia(), testing edge cases, cases that should throw an error, and 
that cases with already known outcomes return the correct values

In [14]:
def test_returns_empty_table_when_range_is_zero():
    """psr_to_gaia() returns empty table when height and width equal zero

    Tests that, when given an otherwise valid dataset, if height and width are zero, an empty astropy Table
    is returned.
    """

    jname = 'J1012+5307'
    raj = '10:12:33.4'
    decj = '+53:07:02.2'
    pmra = '9.240'
    pmdec = '1.770'
    posepoch = '56000.00'
    height = 0.
    width = 0.

    check = psr_to_gaia(jname, raj, decj, pmra, pmdec, posepoch, height, width)
    assert len(check) == 0

INFO: Query finished. [astroquery.utils.tap.core]


In [35]:
def test_minus_sign_in_decj_and_pm_strings():
    """
    Tests that the function psr_to_gaia() still behaves properly with string inputs that are intended to be 
    interpreted as negative values, such as decj and pmdec
    """

    jname = 'J0024-7204J'
    raj = '00:23:59.4'
    decj = '-72:03:58.7'
    pmra = '5.270'
    pmdec = '-3.590'
    posepoch = '51600.00'
    height = 1.
    width = 1.

    check = psr_to_gaia(jname, raj, decj, pmra, pmdec, posepoch, height, width)
    astropy_table = parse_single_table('J0024-7204J_matches.vot.gz').to_table(use_names_over_ids=True)
    check.info()
    # assert check == astropy_table

test_minus_sign_in_decj_and_pm_strings()

-9.972222222222224e-07 deg / yr


TypeError: 'CompositeUnit' object is not callable

In [19]:
astropy_table = parse_single_table('J0024-7204J_matches.vot.gz').to_table(use_names_over_ids=True)
astropy_table

source_id,ra,ra_error,dec,dec_error,parallax,parallax_error,phot_g_mean_mag,bp_rp,radial_velocity,radial_velocity_error,phot_variable_flag,teff_val,a_g_val
,deg,mas,deg,mas,mas,mas,mag,mag,km / s,km / s,,K,mag
int64,float64,float64,float64,float64,float64,float64,float32,float32,float64,float64,object,float32,float32
4689638059950659584,5.97241862484132,1.1218197395986786,-72.07724313868937,1.1522017052178715,--,--,12.169037,1.3813763,--,--,NOT_AVAILABLE,4485.0,--
4689638059950659328,5.972237513069744,0.05401054600347419,-72.06867647066856,0.03838187634966246,0.17901260184049259,0.05563867117858709,12.772815,1.3368607,--,--,NOT_AVAILABLE,4675.85,--
4689638064247508992,5.959260732208984,0.050964009928267606,-72.07060764075895,0.04498876408737587,0.2985669295192851,0.0718588865764239,13.823187,0.92626095,--,--,NOT_AVAILABLE,5286.9,--
4689638059950658816,5.971585098360353,0.03065644558825252,-72.0710082981874,0.022933079906530776,0.20389835991167002,0.032139191567241564,10.900701,2.080802,-28.410603007965378,0.883241520872489,NOT_AVAILABLE,4012.9749,0.97
4689638059950657920,5.97012969664121,0.053846396343092656,-72.07591033878539,0.04294341249822893,0.07673052292975972,0.0637344759189761,12.297852,1.4347897,--,--,NOT_AVAILABLE,4610.0,--
4689637991231188352,5.982917396489215,0.24869482903968546,-72.08175889572875,0.18126983312158823,--,--,12.755561,1.234786,--,--,NOT_AVAILABLE,5143.45,--
4689638059950664064,5.980239570245464,0.047467639866324185,-72.06722400614316,0.07365683978439337,--,--,13.760385,1.1834755,--,--,NOT_AVAILABLE,5000.0,--
4689638064253888128,5.9650780705944655,0.17688341657523676,-72.07272867864056,0.4782671499353005,-0.862823191245309,0.45927497629857633,12.792448,1.3512039,--,--,NOT_AVAILABLE,4733.29,--
